# Comparing the three algorithms by Neal

In [ ]:
import numpy as np
import scipy.stats as stats
import subprocess
import matplotlib.pyplot as plt
# import pip
# pip.main(["install", packagename])

In [ ]:
# Utility to save files with Unix-like newlines
def save_np(filename, npobj):
    with open(filename, 'wb') as f:
        np.savetxt(f, npobj, fmt='%1.5f')

In [ ]:
# Generate data
rng = 20201124
np.random.seed(rng)
n = 200
norm1 = np.random.normal(loc=-4.0, scale=1.0, size=int(n/2))
norm2 = np.random.normal(loc=+4.0, scale=1.0, size=int(n/2))
data_uni = np.concatenate((norm1, norm2))
# Generate grid
grid_uni = np.arange(-10, +10, 0.1)

In [ ]:
# Save to file
save_np("../resources/csv/in/data_uni.csv", data_uni)
save_np("../resources/csv/in/grid_uni.csv", grid_uni)

In [ ]:
# True density of data
true_pdf = 0.5 * stats.norm.pdf(grid_uni, -4.0, 1.0) + \
           0.5 * stats.norm.pdf(grid_uni, +4.0, 1.0)

In [ ]:
# Define list of algorithms
algos = ["Neal2", "Neal3", "Neal8"]

In [ ]:
# Run the executable
for algo in algos:
    cmd = ["../build/run",
        algo, str(rng), "0", "1000", "100",
        "NNIG", "../resources/asciipb/nnig_ngg_prior.asciipb",
        "DP",   "../resources/asciipb/dp_gamma_prior.asciipb",
        f"../{algo}.recordio",
        "../resources/csv/in/data_uni.csv",
        "../resources/csv/in/grid_uni.csv",
        f"../resources/csv/out/uni_{algo}_dens.csv",
        f"../resources/csv/out/uni_{algo}_mass.csv",
        f"../resources/csv/out/uni_{algo}_nclu.csv",
        f"../resources/csv/out/uni_{algo}_clus.csv"
    ]
    subprocess.run(cmd, capture_output=True)

In [ ]:
# Densities
plt.figure(figsize=(16, 8))
for algo in algos:
    matr = np.genfromtxt(f"../resources/csv/out/uni_{algo}_dens.csv", delimiter=',')
    plt.plot(grid_uni, np.exp(np.mean(matr, axis=0)))
plt.plot(grid_uni, true_pdf, color="red", linestyle="--")
plt.legend(algos + ["true"])
plt.title("Univariate densities")